### Lemmatiser les textes
#### Textes anglais
Utilisation du module NLTK.

#### Textes français
Nous avons lemmatizer avec SPACY car nous n'avons pas réussi à prendre en mains le module FrenchLefffLemmatizer.

In [1]:
import glob
import spacy

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Téléchargement des ressources nécessaires
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

# Charger le modèle de langue française
nlp = spacy.load('fr_core_news_sm')

def wordnet_pos(treebank_tag: str) -> str:
    """
    Convertit les étiquettes de partie du discours de Treebank en étiquettes de WordNet.   
    Args:
        treebank_tag (str): L'étiquette de partie du discours de Treebank.       
    Returns:
        str: L'étiquette de partie du discours correspondante pour WordNet.
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatizer_nltk(text: str) -> str:
    """
    Lemmatiser un texte en anglais en utilisant NLTK.
    Args:
        text (str): Le texte à lemmatiser.    
    Returns:
        str: Le texte lemmatisé.
    """
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, wordnet_pos(pos)) for token, pos in pos_tags]
    return ' '.join(lemmatized_tokens)

def generate_path(path: str) -> str:
    """
    Génère un nouveau chemin de fichier en remplaçant 'tok' par 'lem' dans le nom du fichier.
    Args:
        path (str): Le chemin du fichier d'origine.
        
    Returns:
        str: Le nouveau chemin de fichier avec 'lem' au lieu de 'tok'.
    """
    print(f"Chemin d'origine : {path}")
    
    # Diviser le chemin du fichier pour manipuler le nom de fichier
    path_parts = path.split("_")
    
    # Modifier la partie finale du nom de fichier pour remplacer 'tok' par 'lem'
    end = path_parts[2]
    end_parts = end.split(".")
    end_parts[1] = "lem"
    new_end = ".".join(end_parts)
    path_parts[2] = new_end
    
    # Reconstituer le nouveau chemin de fichier
    new_path = "_".join(path_parts)
    print(f"Nouveau chemin : {new_path}")
    return new_path

def lemmatizer_spacy(text: str) -> str:
    """
    Lemmatiser un texte en français en utilisant spaCy.
    Args:
        text (str): Le texte à lemmatiser.  
    Returns:
        str: Le texte lemmatisé.
    """
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return ' '.join(lemmatized_tokens)

def correcter(text: str) -> str:
    """
    Corrige les entités HTML dans un texte. 
    Args:
        text (str): Le texte contenant des entités HTML.    
    Returns:
        str: Le texte avec les entités HTML corrigées.
    """
    html_entities = {
        "&apos;": "'",
        "&quot;": '"',
        "&amp;": "&",
        "&lt;": "<",
        "&gt;": ">"
    }
    for entity, char in html_entities.items():
        text = text.replace(entity, char)
    return text

def lemmatization(paths: str) -> None:
    """
    Lemmatiser les fichiers texte en français ou en anglais.
    
    Args:
        paths (str): Liste des chemins de fichiers à lemmatiser.
        
    Returns:
        None
    """
    for path in paths:
        lemmatize_text = ""
        new_path = generate_path(path)
        
        # Lire le contenu du fichier d'origine
        with open(path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            for line in lines:
                # Corriger les entités HTML pour chaque ligne et lemmatiser
                corrected_line = correcter(line.strip())
                if path[-2:] == "fr":
                    lemmatized_line = lemmatizer_spacy(corrected_line)
                elif path[-2:] == "en":
                    lemmatized_line = lemmatizer_nltk(corrected_line)
                lemmatize_text += lemmatized_line + "\n"  # Retour à la ligne après chaque ligne lemmatisée
        
        # Afficher le texte lemmatisé
        #print(lemmatize_text)

        # Écrire le texte lemmatisé dans le nouveau fichier
        with open(new_path, "w", encoding="utf-8") as new_file:
            new_file.write(lemmatize_text)


[nltk_data] Downloading package punkt to /home/guilhem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/guilhem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/guilhem/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/guilhem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Utiliser glob pour trouver tous les fichiers
file_paths = glob.glob('../data/clean/europarl/*')

paths = []
for file in file_paths:
    file_parts = file.split("_")
    if file_parts[1] in ["train", "dev", "test"]:
        if file_parts[2] in ["10k.tok.true.clean.fr", "1k.tok.true.clean.fr", "10k.tok.true.clean.en", "1k.tok.true.clean.en", "500.tok.true.clean.fr", "500.tok.true.clean.en", "50.tok.true.clean.fr", "50.tok.true.clean.en"]:
            paths.append(file)

lemmatization(paths)

Chemin d'origine : ../data/clean/europarl/Europarl_test_500.tok.true.clean.en
Nouveau chemin : ../data/clean/europarl/Europarl_test_500.lem.true.clean.en
Chemin d'origine : ../data/clean/europarl/Europarl_dev_1k.tok.true.clean.fr
Nouveau chemin : ../data/clean/europarl/Europarl_dev_1k.lem.true.clean.fr
Chemin d'origine : ../data/clean/europarl/Europarl_train_10k.tok.true.clean.fr
Nouveau chemin : ../data/clean/europarl/Europarl_train_10k.lem.true.clean.fr
Chemin d'origine : ../data/clean/europarl/Europarl_dev_1k.tok.true.clean.en
Nouveau chemin : ../data/clean/europarl/Europarl_dev_1k.lem.true.clean.en
Chemin d'origine : ../data/clean/europarl/Europarl_test_500.tok.true.clean.fr
Nouveau chemin : ../data/clean/europarl/Europarl_test_500.lem.true.clean.fr
Chemin d'origine : ../data/clean/europarl/Europarl_train_10k.tok.true.clean.en
Nouveau chemin : ../data/clean/europarl/Europarl_train_10k.lem.true.clean.en
